# k nearest neighbor
### import libs

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

### bereinigen

In [2]:
def price_to_float(price):
    price = float(price.replace('$','').replace(',',''))
    return price

In [3]:
listings = pd.read_csv('~/studies/analytics/listings.csv')
original = listings.copy()
listings.dropna()
listings.shape
listings['price'] = listings['price'].map(price_to_float)
listings['price']
df = listings[['beds', 'bedrooms', 'room_type', 'price']]
df.dropna()

,beds,bedrooms,room_type,price
0,1.0,1.0,Private room,21.0
1,2.0,1.0,Entire home/apt,60.0
2,2.0,1.0,Entire home/apt,90.0
3,1.0,1.0,Private room,28.0
4,1.0,1.0,Entire home/apt,125.0
...,...,...,...,...
24417,1.0,1.0,Private room,54.0
24418,0.0,0.0,Entire home/apt,60.0
24419,1.0,0.0,Private room,36.0
24420,2.0,1.0,Private room,50.0


In [4]:
df = df[df['beds'].notna() == True]
df = df[df['bedrooms'].notna() == True]

In [5]:
X = df[['beds', 'bedrooms', 'room_type']]
y = df['price'].values
type(y)

numpy.ndarray

In [6]:
X.dropna()

,beds,bedrooms,room_type
0,1.0,1.0,Private room
1,2.0,1.0,Entire home/apt
2,2.0,1.0,Entire home/apt
3,1.0,1.0,Private room
4,1.0,1.0,Entire home/apt
...,...,...,...
24417,1.0,1.0,Private room
24418,0.0,0.0,Entire home/apt
24419,1.0,0.0,Private room
24420,2.0,1.0,Private room


In [7]:
le_room_type = LabelEncoder()
X['room_type'] = le_room_type.fit_transform(X.room_type)

/Users/d064689/studies/analytics/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
enc = preprocessing.OneHotEncoder()
room_type = [X['room_type'].values]
enc.fit(room_type)

/Users/d064689/studies/analytics/analytics/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [9]:
onehotlabels = enc.transform(room_type).toarray()
onehotlabels.shape

(1, 24370)

In [10]:
X['room_type']= onehotlabels[0]

/Users/d064689/studies/analytics/analytics/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
beds_train, beds_test, y_train, y_test = train_test_split(df.beds, df.price, test_size=0.7, random_state=0)

In [12]:
min_max_scaler = preprocessing.MinMaxScaler()
beds_train_scaled = min_max_scaler.fit_transform(beds_train.to_numpy().reshape(-1,1))

In [13]:
y_train_scaled = min_max_scaler.fit_transform([y_train])

In [14]:
knn = KNeighborsRegressor(n_neighbors=2)

In [15]:
knn.fit(beds_train_scaled, y_train_scaled[0])

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                    weights='uniform')

In [16]:
knn.predict([[3]])

array([0.])